In [10]:
from langchain_chroma import Chroma

## LangSmith 설정

In [15]:
import os
import getpass
from dotenv import load_dotenv

load_dotenv()

langchain_tracing = os.getenv("LANGCHAIN_TRACING_V2")
langchain_api_key = os.getenv("LANGCHAIN_API_KEY")
aws_region = os.getenv("AWS_DEFAULT_REGION")
tavily_api_key = os.getenv("TAVILY_API_KEY")

In [12]:
from langchain_aws import ChatBedrock
from langchain_core.messages import HumanMessage

# Bedrock 모델 설정
model = ChatBedrock(
    model_id="anthropic.claude-3-5-sonnet-20240620-v1:0",
    model_kwargs=dict(temperature=0),
    region_name='us-east-1'
)

### 필요한정보 = 가지고있는정보.추출()
### 필요한정보 = 추출(가지고있는정보, 알고있는정보)

# 모델 호출
model.invoke([HumanMessage(content="다음 미국 대선 후보는 누구인가요?")])


AIMessage(content='2024년 미국 대선의 주요 후보들은 아직 확정되지 않았습니다. 하지만 현재 상황을 보면:\n\n1. 민주당:\n   - 조 바이든 현 대통령이 재선에 도전할 가능성이 높습니다.\n\n2. 공화당:\n   - 도널드 트럼프 전 대통령이 출마를 선언했습니다.\n   - 론 드샌티스 플로리다 주지사가 유력한 후보로 거론되고 있습니다.\n   - 니키 헤일리 전 유엔 대사도 출마를 선언했습니다.\n\n그 외에도 여러 정치인들이 출마를 고려하고 있거나 선언할 수 있습니다. 공식적인 후보 등록과 예비선거 과정을 거쳐 각 당의 최종 후보가 결정될 것입니다. 대선은 2024년 11월에 실시될 예정입니다.\n\n상황은 계속 변할 수 있으므로, 최신 정보를 확인하는 것이 중요합니다.', additional_kwargs={'usage': {'prompt_tokens': 27, 'completion_tokens': 358, 'total_tokens': 385}, 'stop_reason': 'end_turn', 'model_id': 'anthropic.claude-3-5-sonnet-20240620-v1:0'}, response_metadata={'usage': {'prompt_tokens': 27, 'completion_tokens': 358, 'total_tokens': 385}, 'stop_reason': 'end_turn', 'model_id': 'anthropic.claude-3-5-sonnet-20240620-v1:0'}, id='run-e7ad9df2-a313-48d7-841e-03656d7c5005-0', usage_metadata={'input_tokens': 27, 'output_tokens': 358, 'total_tokens': 385})

### 문서(Document) 생성

In [13]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="개는 충성스럽고 친절한 동반자로 알려져 있습니다.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="고양이는 독립적인 반려동물로 자신의 공간을 즐기는 경향이 있습니다.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="금붕어는 초보자에게 인기가 많은 반려동물로, 관리가 비교적 간단합니다.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="앵무새는 인간의 말을 흉내낼 수 있는 지능적인 새입니다.",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="토끼는 사회적인 동물로, 뛰어놀 공간이 많이 필요합니다.",
        metadata={"source": "mammal-pets-doc"},
    ),
]


### 벡터 스토어

#### 벡터 스토어 설정

In [16]:
from langchain_chroma import Chroma
from langchain_aws import BedrockEmbeddings

# Amazon Titan 임베딩 모델 사용
embedding_model = BedrockEmbeddings(
    model_id='amazon.titan-embed-text-v1',
)

# 벡터 스토어 생성
vectorstore = Chroma.from_documents(
    documents,
    embedding=embedding_model,
)

In [17]:
embedding = embedding_model.embed_query("고양이")
embedding

[1.1171875,
 0.3671875,
 0.1826171875,
 0.33984375,
 -0.177734375,
 -0.345703125,
 0.162109375,
 -0.0010528564453125,
 -0.31640625,
 -1.140625,
 -0.138671875,
 0.45703125,
 -0.26171875,
 0.11962890625,
 0.703125,
 0.0301513671875,
 0.0380859375,
 -1.671875,
 -1.1796875,
 0.39453125,
 -0.19140625,
 -0.39453125,
 -1.3359375,
 1.609375,
 0.25,
 0.67578125,
 0.7734375,
 -1.4765625,
 -0.578125,
 -1.265625,
 0.80078125,
 0.08935546875,
 -0.67578125,
 -1.9921875,
 -0.2099609375,
 -0.412109375,
 0.1513671875,
 -1.140625,
 -0.07080078125,
 0.486328125,
 -0.6015625,
 0.205078125,
 1.390625,
 -0.291015625,
 0.453125,
 -0.1435546875,
 0.28515625,
 0.458984375,
 0.0478515625,
 -0.384765625,
 0.259765625,
 1.2890625,
 -1.09375,
 -0.83984375,
 0.041748046875,
 0.53515625,
 -1.328125,
 -0.171875,
 -1.40625,
 0.024658203125,
 0.3515625,
 0.71484375,
 0.359375,
 0.796875,
 0.4296875,
 -0.5625,
 0.00102996826171875,
 -0.10693359375,
 -0.6640625,
 -0.3359375,
 -0.26953125,
 -0.259765625,
 1.0703125,
 0.13

### 유사도 기반 문서 검색

In [18]:
# "고양이"와 유사한 문서 검색
result = vectorstore.similarity_search("고양이")
for doc in result:
    print(doc.page_content)

고양이는 독립적인 반려동물로 자신의 공간을 즐기는 경향이 있습니다.
금붕어는 초보자에게 인기가 많은 반려동물로, 관리가 비교적 간단합니다.
개는 충성스럽고 친절한 동반자로 알려져 있습니다.
앵무새는 인간의 말을 흉내낼 수 있는 지능적인 새입니다.


In [19]:
result

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='고양이는 독립적인 반려동물로 자신의 공간을 즐기는 경향이 있습니다.'),
 Document(metadata={'source': 'fish-pets-doc'}, page_content='금붕어는 초보자에게 인기가 많은 반려동물로, 관리가 비교적 간단합니다.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='개는 충성스럽고 친절한 동반자로 알려져 있습니다.'),
 Document(metadata={'source': 'bird-pets-doc'}, page_content='앵무새는 인간의 말을 흉내낼 수 있는 지능적인 새입니다.')]

### 유사도 점수 포함 검색

In [21]:
result_with_score = vectorstore.similarity_search_with_score("고양이")
for doc, score in result_with_score:
    print(f"문서: {doc.page_content}, 유사도 점수: {score}")

문서: 고양이는 독립적인 반려동물로 자신의 공간을 즐기는 경향이 있습니다., 유사도 점수: 423.80316162109375
문서: 금붕어는 초보자에게 인기가 많은 반려동물로, 관리가 비교적 간단합니다., 유사도 점수: 518.2442016601562
문서: 개는 충성스럽고 친절한 동반자로 알려져 있습니다., 유사도 점수: 533.6543579101562
문서: 앵무새는 인간의 말을 흉내낼 수 있는 지능적인 새입니다., 유사도 점수: 575.2368774414062


### 벡터로 쿼리 검색

In [22]:
# 쿼리를 벡터로 변환하여 검색
embedidng = embedding_model.embed_query("고양이")
vector_result = vectorstore.similarity_search_by_vector(embedding)
for doc in vector_result:
    print(doc.page_content)

고양이는 독립적인 반려동물로 자신의 공간을 즐기는 경향이 있습니다.
금붕어는 초보자에게 인기가 많은 반려동물로, 관리가 비교적 간단합니다.
개는 충성스럽고 친절한 동반자로 알려져 있습니다.
앵무새는 인간의 말을 흉내낼 수 있는 지능적인 새입니다.


## 리트리버(Retrievers)

### 간단한 리트리버 생성

In [23]:
from langchain_core.runnables import RunnableLambda

# 유사도 검색을 통해 상위 1개의 결과를 반환하는 리트리버 생성
retriever = RunnableLambda(vectorstore.similarity_search).bind(k=1)

# 리트리버로 "고양이", "상어" 쿼리 처리
results = retriever.batch(["고양이", "상어"])
for result in results:
    for doc in result:
        print(doc.page_content)

고양이는 독립적인 반려동물로 자신의 공간을 즐기는 경향이 있습니다.
앵무새는 인간의 말을 흉내낼 수 있는 지능적인 새입니다.


### 리트리버로 RAG 응용 프로그램 생성

#### RAG 체인 생성

In [24]:
from langchain_aws import ChatBedrock
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# Amazon Bedrock LLM 설정 (Claude 모델 사용)
llm = ChatBedrock(
    model_id="anthropic.claude-3-5-sonnet-20240620-v1:0"
)
# 프롬프트 템플릿 정의
message = """
질문에 주어진 문맥만을 사용하여 답변하세요.

질문:
{question}

문맥:
{context}
"""
prompt = ChatPromptTemplate.from_messages([("human", message)])

# RAG 체인 설정: 리트리버로 문맥을 가져와 질문과 함께 LLM에 전달
rag_chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | llm

# 질문에 대한 답변 생성
response = rag_chain.invoke("고양이에 대해 알려주세요")  # 위 RunnablePassthrough()에 이 입력이 들어감
print(response.content)

주어진 문맥에 따르면, 고양이는 독립적인 반려동물입니다. 그들은 자신만의 공간을 즐기는 경향이 있습니다. 이는 고양이가 혼자 있는 시간을 좋아하고 자율성을 중요시한다는 것을 시사합니다.
